In [2]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment, Environment, ScriptRunConfig, Run 
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget

from azureml.widgets import RunDetails

from azureml.core.webservice import AciWebservice #Webservice, AksWebservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies 

from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, uniform, PrimaryMetricGoal


# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.36.0


# Connect to Workspace

In [3]:
ws = Workspace(workspace_name="Cybertron",resource_group="Cybertron-RG",subscription_id="431dbae5-40ca-438a-8daf-77d7d5580b41")
print("Workspace:",ws.name)

# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name="OptimusPrime")
print("Compute Target:",compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:",ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='Simpsons-PyTorch')
print("Experiment:",exp.name)

Workspace: Cybertron
Compute Target: OptimusPrime
Datastore: workspaceblobstore
Experiment: Simpsons-PyTorch


# Remote training

In [4]:
simpsons_ds = Dataset.get_by_name(ws, name='LegoSimpsons-v3')
print("Dataset:",simpsons_ds.name,"Version:",simpsons_ds.version)

Dataset: LegoSimpsons-v3 Version: 3


In [5]:
curated_env_name = 'AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = pytorch_env.clone(new_name='pytorch-simpson-training')
print("Environment:",pytorch_env.name)

Environment: pytorch-simpson-training


In [7]:
args = [
    '--data-folder', simpsons_ds.as_named_input('simpsons').as_mount(),
    '--num-epochs', 10,
    '--model-name', 'Simpons-Classification'
]

config = ScriptRunConfig(
    source_directory = '../scripts/train', 
    script = 'train.py', 
    compute_target = compute_target,
    environment = pytorch_env,
    arguments=args,
)

In [8]:
run = exp.submit(config)
print("Run:",run.id)

Run: Simpsons-PyTorch_1636619021_acddd04e


In [9]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
model = run.register_model(
   model_name='Simpsons-Classification',
   model_path='outputs',
   model_framework='PyTorch',
   model_framework_version='1.6',
   description="Simpsons PyTorch Classifier (From Jupyter Notebook)",
   tags={'Conference':''},
)

print("Model '{}' version {} registered ".format(model.name,model.version))

Model 'Simpsons-Classification' version 2 registered 


# Test deployment to an Azure Container Instance

In [11]:
myenv = Environment(name="simpsons-inference")

conda_dep = CondaDependencies()

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("pillow==5.4.1")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

In [12]:
inference_config = InferenceConfig(
    entry_script = "../scripts/score/score.py", 
    environment = myenv
)

In [13]:
deploy_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 4,
    description='Simpson Lego Classifier'
)

In [16]:
aci_service = Model.deploy(ws, 
    name="simpsons-classification-test", 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deploy_config, 
    overwrite = True
)

aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-11 09:47:12+01:00 Creating Container Registry if not exists.
2021-11-11 09:47:14+01:00 Use the existing image.
2021-11-11 09:47:14+01:00 Generating deployment configuration.
2021-11-11 09:47:15+01:00 Submitting deployment to compute.
2021-11-11 09:47:18+01:00 Checking the status of deployment simpsons-classification-test..
2021-11-11 09:47:50+01:00 Checking the status of inference endpoint simpsons-classification-test.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [17]:
img_name = "https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Bart.jpg"
result = aci_service.run(input_data=json.dumps({ "image": img_name}))

print(result)

{'time': '0.578239', 'prediction': 'Bart-Simpson', 'scores': {'Bart-Simpson': '95.95'}}


# Optimize the model

In [34]:
param_sampling = RandomParameterSampling( {
        'num-epochs': choice(range(3,5)),
        'learning_rate': uniform(0.001, 0.005),
        'momentum': uniform(0.9, 0.99)
    }
)

args = [
    '--data-folder', simpsons_ds.as_named_input('simpsons').as_mount(),
]

script_run_config = ScriptRunConfig(
    source_directory = "../scripts/train", 
    script = 'train.py', 
    compute_target=compute_target,
    environment = pytorch_env,
    arguments=args,
)

hd_config = HyperDriveConfig(run_config = script_run_config,
                             hyperparameter_sampling = param_sampling,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=1,
                             max_concurrent_runs=1)

In [35]:
hyperdrive_run = exp.submit(hd_config)

In [36]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [3]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])
print('Learning rate:', parameter_values[3])
print('Momentum:', parameter_values[5])

NameError: name 'hyperdrive_run' is not defined

In [2]:
model = best_run.register_model(
    model_name='Simpsons-Classification',
    model_path='outputs',
    model_framework='PyTorch',
    model_framework_version='1.1',
    description="Simpsons best model",
    tags={'Conference':'Test 1','HPT':'true'},
)

print("Model '{}' version {} registered ".format(model.name,model.version))

NameError: name 'best_run' is not defined